# Hyperparameter Tuning using HyperDrive

### Import libraries

In [1]:
from azureml.core import Workspace, Experiment

### Create workspace and experiment instances

In [2]:
ws = Workspace.from_config()
experiment_name = 'capstone-spam-classification-experiment'

experiment=Experiment(ws, experiment_name)

### Connect to compute cluster target

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
cluster_name = "capstone-compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Get Data

In [4]:
from azureml.core import Dataset
training_dataset = Dataset.get_by_name(ws, name='capstone-spam-dataset')

### Hyperdrive Configuration
We are using a Logistic Regression classifier, the main hyperparameters there are C and max_iter. 

In [5]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
from azureml.train.hyperdrive.policy import BanditPolicy
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.sampling import RandomParameterSampling
param_sampling = RandomParameterSampling( 
    {
        "C": choice(0.01, 0.001, 0.1, 1),
        "max_iter": choice(50,100),
    }
)

# Setup environment for your training run
from azureml.core import Environment
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')


# Create a ScriptRunConfig Object to specify the configuration details of your training job
from azureml.core import ScriptRunConfig
script_run_config = ScriptRunConfig(source_directory=".",
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create Hyperdrive config
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
hyperdrive_run_config = HyperDriveConfig(run_config=script_run_config,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name='Accuracy',
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=10,
                             max_concurrent_runs=4)

In [6]:
# Submit your hyperdrive run to the experiment 
hyperdrive_run = experiment.submit(hyperdrive_run_config)

### Run Details

In [7]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show() 
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_fe6d9b1a-e654-484c-a58d-5260b5f4e533
Web View: https://ml.azure.com/runs/HD_fe6d9b1a-e654-484c-a58d-5260b5f4e533?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-208297/workspaces/quick-starts-ws-208297&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-09-22T09:04:31.104907][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-09-22T09:04:31.8516670Z][SCHEDULER][INFO]Scheduling job, id='HD_fe6d9b1a-e654-484c-a58d-5260b5f4e533_0' 
[2022-09-22T09:04:31.9915447Z][SCHEDULER][INFO]Scheduling job, id='HD_fe6d9b1a-e654-484c-a58d-5260b5f4e533_1' 
[2022-09-22T09:04:32.1066085Z][SCHEDULER][INFO]Scheduling job, id='HD_fe6d9b1a-e654-484c-a58d-5260b5f4e533_2' 
[2022-09-22T09:04:32.1670256Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_fe6d9b1a-e654-484c-a58d-5260b5f4e533_0' 
[2022-09-22T09:04:32.179211][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to t

{'runId': 'HD_fe6d9b1a-e654-484c-a58d-5260b5f4e533',
 'target': 'capstone-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-09-22T09:04:30.442233Z',
 'endTimeUtc': '2022-09-22T09:08:05.605805Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'c0954b2d-740f-4d91-b1dd-3aa9da7b55c5',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.44.0',
  'space_size': '8',
  'score': '0.9691535150645624',
  'best_child_run_id': 'HD_fe6d9b1a-e654-484c-a58d-5260b5f4e533_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_fe6d9b1a-e654-484c-a58d-5260b5f4e533_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': 

KeyError: 'log_files'

### Get the best model and display its properties

In [9]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Best Accuracy:', best_run_metrics['Accuracy'])

Best Run Id:  HD_fe6d9b1a-e654-484c-a58d-5260b5f4e533_0

 Best Accuracy: 0.9691535150645624


### Register the model

In [17]:
model = best_run.register_model(
    model_name="hyperdrive-best-model", model_path="."
)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

